In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 대기시간 설정을 위한 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.parse import quote
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
from tqdm import tqdm

# 시스템 확인 - 파일 경로 확인
import os
# 다운로드 파일이동
import shutil

#엑셀파일 불러들이기 위한
import xlrd

#time.sleep을 위한 난수생성
import random

import pandas as pd

### 제주도 한달살기, 한달살이 - 블로그 데이터 확인


In [12]:
# 제주 + "제주낭만펜션" 검색결과 URL
# "" 포함시 필수검색 옵션
url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query=%EC%A0%9C%EC%A3%BC+%22%EB%82%AD%EB%A7%8C%EC%A0%9C%EC%A3%BC%ED%8E%9C%EC%85%98%22+-%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C+-%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C&tqi'

In [21]:
def convert_date(s):
    if len(s) == 11 : # 2023.12.13.  
        return s[:-1] # 마지막 점 제거
    
    now = datetime.now()
    if '어제' in s:
        date = now - timedelta(days = 1)
    elif '일' in s:
        date = now - timedelta(days = int(s[0]))
    elif '시간' in s:
        date = now - timedelta(hours = int(s[:-4]))
    elif '분' in s:
        date = now - timedelta(minutes = int(s[:-3]))
    elif '주' in s:
        date = now - timedelta(weeks = int(s[:-3]))

    return date.strftime('%Y.%m.%d')

In [48]:
base_url = 'https://search.naver.com/search.naver?where=blog&&sm=tab_jum&query='

keyword_list = ['제주도', '한달살기']
keyword_search = ''

for word in keyword_list:    
    keyword_search += f'+{quote(word)}'

# 제외하고자 하는 검색어
# 파트너스 - -%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4
# 예약코드 - -%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C
delete_words = ["파트너스", "예약코드", '부동산', '이벤트', '리브애니웨어', '미스터멘션', '탁송', '퇴실청소']
delete_words

delete_search = ""
for word in delete_words:    
    delete_search += f'+-{quote(word)}'

# 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
url = base_url + keyword_search + delete_search
driver = webdriver.Chrome()
driver.get(url)

round(random.uniform(1.5, 2.5), 3)

while True:
    last_height = driver.execute_script("return document.body.scrollHeight")

    # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
    driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
    time.sleep(round(random.uniform(0.0, 1.0), 3))
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
    time.sleep(round(random.uniform(1.0, 1.5), 3))

    # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break

round(random.uniform(4.5, 6.5), 3)

soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select_one('.lst_view').select('.view_wrap')

result = []

for li in lis:
    try :
        title = li.select_one('.title_area').select_one('a').text.strip()
        #title = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", title_raw)
    except:
        title = 'error'
    
    try :
        content = li.select_one('.dsc_link').text.strip()
        #content = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", content_raw)
    except :
        content = 'error'
    try :          
        blog_link = li.select_one('.title_area').select_one('a')['href']        
    except :
        blog_link = 'error'

    try :          
        naver_time = li.select_one('.sub').text.strip()
        blog_time = convert_date(naver_time)
    except :
        blog_time = 'error'

    if blog_link.find('blog.naver.com') > 0:
                            
        driver = webdriver.Chrome()
        driver.get(blog_link)
        driver.switch_to.frame('mainFrame')
        time.sleep(round(random.uniform(0.0, 1.2), 3))

        
        lis = driver.find_element(By.CLASS_NAME, 'se-main-container').find_elements(By.CLASS_NAME, 'se-text')
        blog_contents = []

        for li in lis:
            #sub_content = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", li.text.strip())

            try : 
                sub_content = li.text.strip()
                blog_contents.append(sub_content.replace('\U0001f979','').replace('\n',''))
                blog_text = ' '.join(blog_contents)
            except :
                blog_text = 'error'

        time.sleep(round(random.uniform(0.5, 1.5), 3))

   
    result.append({'title' : title,
                'contents' : content,
                'time' : blog_time,
                'blog_text' : blog_text})


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.200)
Stacktrace:
	GetHandleVerifier [0x00007FF7B9F02142+3514994]
	(No symbol) [0x00007FF7B9B20CE2]
	(No symbol) [0x00007FF7B99C76AA]
	(No symbol) [0x00007FF7B99A0AFD]
	(No symbol) [0x00007FF7B9A3CB1B]
	(No symbol) [0x00007FF7B9A5218F]
	(No symbol) [0x00007FF7B9A35D93]
	(No symbol) [0x00007FF7B9A04BDC]
	(No symbol) [0x00007FF7B9A05C64]
	GetHandleVerifier [0x00007FF7B9F2E16B+3695259]
	GetHandleVerifier [0x00007FF7B9F86737+4057191]
	GetHandleVerifier [0x00007FF7B9F7E4E3+4023827]
	GetHandleVerifier [0x00007FF7B9C504F9+689705]
	(No symbol) [0x00007FF7B9B2C048]
	(No symbol) [0x00007FF7B9B28044]
	(No symbol) [0x00007FF7B9B281C9]
	(No symbol) [0x00007FF7B9B188C4]
	BaseThreadInitThunk [0x00007FFD95A47344+20]
	RtlUserThreadStart [0x00007FFD95B826B1+33]


In [49]:
result = pd.DataFrame(result)
result

,title,contents,time,blog_text
0,돌아기랑 제주도한달살기 24일째(406일) : 비양도,첨에 제주도 한달살기 올때도 어디갈지 생각을 안하고 와서 매번 전날이나 아침에 오늘...,2023.11.27,첨에 제주도 한달살기 올때도 어디갈지 생각을 안하고 와서 매번 전날이나 아침에 오늘...
1,"제주도 한달살기 (20) - 우도 코코나라, 문빵구, 돌담집, 몬딱...",한달살기 중 두번째 우도 ㅎㅎ 처음은 전기차를 빌렸지만 이번엔 카트형식으로 나란히 ...,2023.11.07,한달살기 중 두번째 우도 ㅎㅎ처음은 전기차를 빌렸지만 이번엔 카트형식으로 나란히 앉...
2,제주동쪽숙소 제주도한달살기하기 좋은 구좌 제주오션뷰2차♪,#제주오션뷰2차 #제주도펜션 #제주동쪽숙소 제주에는 크게 동쪽과 서쪽으로 나뉘면서 ...,2023.12.12,#제주오션뷰2차#제주도펜션#제주동쪽숙소제주에는 크게 동쪽과 서쪽으로 나뉘면서 다양한...
3,제주도 한달살기 비용 (한달살이 생활비),제주도 한달살기는 돈이 적게 들지는 않지만 우리 가족에게 꽤 많은 추억을 남겼으므로...,2022.02.23,"2021년 4월에 제주도 한달살기를 하고 왔다.세식구 기준(어른 둘, 아기 하나)으..."
4,제주도 한달살기 숙소 비용 제주 중문 애견동반 더맨션 이이공공,그중에서도 제일 마음에 들었던 건 오후 시간 깊숙이 들어오는 가을 햇살 창밖에 보이...,2023.10.14,이번에는 좀 여유로운 여행을 하고 싶어서다소 긴 일정으로 제주도에 방문했어요.여행...
...,...,...,...,...
57,제주도 한달살기_협재카페 웨이뷰 & 우린 먹으러 왔나보다 ㅎㅎ,뷰가 그냥 멋있죠 ㅎㅎㅎㅎㅎ 제주도 대가족 한달살기 숙소 <제주아리>에서 찍은 오늘...,2023.07.25,뷰가 그냥 멋있죠 ㅎㅎㅎㅎㅎ제주도 대가족 한달살기 숙소 <제주아리>에서 찍은 오늘의...
58,[제주도 한달살기] Day 16 - 재정비 시간,한달살기의 약 반이 지나가는 시점 오늘은 언니가 가고 저녁에 여동기들이 오기로 한 ...,2023.08.15,한달살기의 약 반이 지나가는 시점오늘은 언니가 가고저녁에 여동기들이 오기로 한 날....
59,제주도 한달살기(6탄)제주바다 사진찍기 좋은곳 광치기해변은...,대형 가오리연을 하늘 높이 날리는 거다. 제주바다 사진찍기 좋은곳 광치기해변! 다음...,2023.11.02,아! 광치기해변에 다시 온 게 몇 년 만이더냐~~~?제주도 사진찍기 좋은곳을 꼽으라...
60,[제주도 한달살기] 6일차,제주도의 #바람의_언덕 주차장의 화산석으로 구운 #오징어 는 강추다. #드라이브 내...,2023.08.26,#제주_자동차여행_코스북 4번 코스 가봄코스는 아래 번호 순서별이다.#성산일출봉 많...


In [51]:
result[result['blog_text'] == 'error']

,title,contents,time,blog_text
61,나의 제주도 한달살기 숙소는 힐링농장 터무니하우스! 서귀포...,제주도 구석구석을 누비고 다니는 것도 모자라 섬 속의 섬까지 넘나들며 행복한 여정을...,2023.11.11,error
